# Detecção de Fraudes no IEEE-CIS Fraud Detection com LSTM no PyTorch

## Inteli - Sistemas de Informação - Programação
- **Professor**👨‍🏫: Jefferson de Oliveira Silva
- **Aluno**👨‍🎓: Pedro de Carvalho Rezende

### Objetivo🚨
Treinar e analisar uma rede neural LSTM, no Keras, utilizando o dataset IEEE-CIS Fraud Detection.

### Instruções📃
Para realizar esta atividade, carregue o dataset IEEE-CIS Fraud Detection. Em seguida, faça uma análise exploratória dos dados para entender as características das transações, como distribuições, correlações e possíveis outliers. Use gráficos e estatísticas descritivas para apoiar sua análise.


Depois, prepare os dados para o treinamento da rede neural LSTM. Isso pode incluir o tratamento de valores ausentes, normalização e a criação de sequências temporais, conforme necessário para o modelo LSTM. Em seguida, defina a arquitetura da rede LSTM, escolhendo o número de camadas, neurônios, e outros hiperparâmetros relevantes.


Treine o modelo utilizando um conjunto de treinamento e valide-o utilizando um conjunto de teste. Aplique métricas como precisão, recall, F1-score e AUC-ROC para avaliar o desempenho do modelo. Durante o treinamento, observe a curva de aprendizado para identificar sinais de overfitting ou underfitting.


Após treinar o modelo, analise os resultados. Compare o desempenho nas diferentes fases do treinamento e identifique qualquer possível overfitting ou outros problemas de ajuste. Discuta as estratégias que você utilizou para melhorar a performance do modelo e os insights que obteve durante o processo.


Por fim, documente todas as etapas realizadas, incluindo a preparação dos dados, a definição da arquitetura do modelo, o treinamento, a validação, e a análise dos resultados.

Entregue o link do caderno `.ipynb` em um repositório GitHub.

## **IMPORTANTE**:
- Este notebook está sendo trabalhado com GPUs.
- Por isso é aplicado cuDF.
- Verifique se você está rodando em um tempo de execução com GPU, pois se não, será necessário pequenas mudanças no código. Além de que irá demorar um pouco mais.

# Implementando cuDF

- O principal motivo de estarmos utilizando a GPU é para acelerar o processamento dos dados, visto que o cuDF é uma biblioteca que permite a manipulação de dados em GPU, o que torna o processamento mais rápido.
- Um exemplo claro disso é percebido no momento de rodar qualquer pre processo dos dados

In [1]:
!nvidia-smi

Tue Sep 24 11:04:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
%load_ext cudf.pandas

# Instalações e Importações

In [3]:
%pip install -q -r requirements.txt
# !pip install imblearn scikeras

In [4]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots

import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM
from tensorflow.keras.metrics import BinaryAccuracy, AUC, Precision, Recall, Accuracy, F1Score
from tensorflow.keras.optimizers import Adam, Lion, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from scikeras.wrappers import KerasClassifier, KerasRegressor

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, make_scorer, accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

In [5]:
# baixando o dataset utilizado
!gdown 1T0sT0pK75enS3FvTEChSSHeMmzQyXbIK

Downloading...
From (original): https://drive.google.com/uc?id=1T0sT0pK75enS3FvTEChSSHeMmzQyXbIK
From (redirected): https://drive.google.com/uc?id=1T0sT0pK75enS3FvTEChSSHeMmzQyXbIK&confirm=t&uuid=3f74b351-8283-42c6-add6-bea3e53d1aeb
To: /content/train_transaction.csv
100% 683M/683M [00:08<00:00, 81.1MB/s]


# Exploratória e tratamento do dataset

O dataset escolhido foi o IEEE-CIS Fraud Detection (https://drive.google.com/file/d/1nriPPuYUMXeB6BkCjz_bQI_45WZfxViC/view?usp=drive_link).

O dataset da competição "IEEE Fraud Detection" no Kaggle é projetado para ajudar a detectar fraudes em transações financeiras. Ele contém várias características e variáveis que podem ser usadas para treinar modelos de machine learning para identificar comportamentos fraudulentos.

**Estrutura do Dataset**

- Transações: O dataset inclui informações sobre transações financeiras, como valores, horários e locais.
- Variáveis: As variáveis são divididas em diferentes categorias, incluindo:
  - Identificadores: IDs únicos para cada transação.
  - Características do usuário: Informações sobre o usuário, como histórico de transações.
  - Características da transação: Detalhes sobre a transação em si, como método de pagamento e valor.

O objetivo principal é prever se uma transação é fraudulenta ou não, utilizando técnicas de aprendizado de máquina. Os participantes da competição são incentivados a explorar diferentes algoritmos e abordagens para melhorar a precisão das previsões.

### Descrição das Colunas do Dataset IEEE-CIS Fraud Detection

- **TransactionDT**: Diferença de tempo em relação a uma data de referência (não é um timestamp real).
- **TransactionAMT**: Valor do pagamento da transação em USD.
- **ProductCD**: Código do produto, referente ao produto de cada transação.
- **card1 - card6**: Informações do cartão de pagamento, como tipo de cartão, categoria do cartão, banco emissor, país, etc.
- **addr**: Endereço.
- **dist**: Distância.
- **P_emaildomain e R_emaildomain**: Domínio de e-mail do comprador e do destinatário.
- **C1-C14**: Contagem, como quantos endereços estão associados ao cartão de pagamento, etc. O significado exato está oculto.
- **D1-D15**: Diferença de tempo, como os dias entre a transação anterior, etc.
- **M1-M9**: Correspondência, como a coincidência entre os nomes no cartão e no endereço, etc.
- **Vxxx**: Features criadas pela Vesta, incluindo classificação, contagem e outras relações de entidades.

### Features Categóricas:
- **ProductCD**
- **card1 - card6**
- **addr1, addr2**
- **P_emaildomain**
- **R_emaildomain**
- **M1 - M9**

### Análises

In [6]:
df = pd.read_csv('train_transaction.csv')
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
df.shape

(590540, 394)

In [8]:
df = df.drop(['P_emaildomain', 'R_emaildomain', 'addr1', 'addr2', 'dist1'], axis=1)

In [9]:
df.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 389 entries, TransactionID to V339
dtypes: float64(373), int64(4), object(12)
memory usage: 1.7+ GB


In [10]:
df['isFraud'].value_counts()

,count
isFraud,
0,569877
1,20663


### Tratamento de texto

In [11]:
object_columns = df.select_dtypes(include=['object'])
object_columns

,ProductCD,card4,card6,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,discover,credit,T,T,T,M2,F,T,None,None,None
1,W,mastercard,credit,None,None,None,M0,T,T,None,None,None
2,W,visa,debit,T,T,T,M0,F,F,F,F,F
3,W,mastercard,debit,None,None,None,M0,T,F,None,None,None
4,H,mastercard,credit,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
590535,W,visa,debit,T,T,T,M0,T,F,F,F,T
590536,W,mastercard,debit,T,F,F,M0,F,T,F,F,F
590537,W,mastercard,debit,T,F,F,None,None,T,None,None,None
590538,W,mastercard,debit,T,T,T,M0,F,T,None,None,None


In [12]:
df['M4'].value_counts()

,count
M4,
M0,196405
M2,59865
M1,52826


In [13]:
for col in ['M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9']:
  df[col] = df[col].map({'T': 1, 'F': 0})
  df[col] = pd.to_numeric(df[col], errors='coerce')

mapping = {'M0': 1, 'M1': 2, 'M2': 3}
df['M4'] = df['M4'].map(mapping)

In [14]:
df['ProductCD'].value_counts()

,count
ProductCD,
W,439670
C,68519
R,37699
H,33024
S,11628


In [15]:
mapping = {'W': 1, 'C': 2, 'R': 3, 'H': 4, 'S': 5}
df['ProductCD'] = df['ProductCD'].map(mapping)
df['ProductCD']

,ProductCD
0,1
1,1
2,1
3,1
4,4
...,...
590535,1
590536,1
590537,1
590538,1


In [16]:
df['card4'].value_counts()

,count
card4,
visa,384767
mastercard,189217
american express,8328
discover,6651


In [17]:
mapping = {'visa': 1, 'mastercard': 2, 'american express': 3, 'discover': 4}
df['card4'] = df['card4'].map(mapping)
df['card4']

,card4
0,4.0
1,2.0
2,1.0
3,2.0
4,2.0
...,...
590535,1.0
590536,2.0
590537,2.0
590538,2.0


In [18]:
df['card6'].value_counts()

,count
card6,
debit,439938
credit,148986
debit or credit,30
charge card,15


In [19]:
df = df[~df['card6'].isin(['debit or credit', 'charge card'])]
mapping = {'debit': 1, 'credit': 2}
df['card6'] = df['card6'].map(mapping)
df['card6']

,card6
0,2.0
1,2.0
2,1.0
3,1.0
4,2.0
...,...
590535,1.0
590536,1.0
590537,1.0
590538,1.0


### Remoção de colunas

In [20]:
# dropando todas as colunas que possuem um null_count maior que 450k de linhas, pois isso representa aprox. 75% do dataset, tirando a valorização no modelo.
null_counts = df.isnull().sum()
columns_to_drop = null_counts[null_counts > 443000].index.tolist()
df = df.drop(columns=columns_to_drop)
df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321
0,2987000,0,86400,68.5,1,13926,NaN,150.0,4,142.0,...,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0
1,2987001,0,86401,29.0,1,2755,404.0,150.0,2,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2987002,0,86469,59.0,1,4663,490.0,150.0,1,166.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2987003,0,86499,50.0,1,18132,567.0,150.0,2,117.0,...,135.0,0.0,0.0,0.0,50.0,1404.0,790.0,0.0,0.0,0.0
4,2987004,0,86506,50.0,4,4497,514.0,150.0,2,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# df_d = df.filter(regex='^D')
# df_menos_colunas = df.drop(columns=df_d.columns)

In [22]:
df.shape

(590495, 222)

In [23]:
df['isFraud'].value_counts()

,count
isFraud,
0,569832
1,20663


# Prepração dos dados para o modelo

- Utilizando a coluna TransactionDT, que é um "timedelta", ou seja, a diferença em segundos em relação a um ponto de referência, vou criar janelas temporais baseadas nessa diferença de tempo

- Como o TransactionDT está em segundos, você pode convertê-lo para dias, horas, ou outro intervalo de tempo que faça sentido para o modelo.

In [55]:
# Verifica a quantidade de valores NaN em cada coluna
nan_counts = df.isnull().sum()

# Imprime as colunas com NaN e a quantidade
print(nan_counts[nan_counts > 0])

# Substitui os valores NaN pela mediana de cada coluna
for column in df.columns:
  if df[column].isnull().any():
    df[column].fillna(df[column].median(), inplace=True)

# Verifica se ainda existem valores NaN
print(df.isnull().sum().sum())

card2    8933
card3    1565
card4    1577
card5    4247
card6    1571
         ... 
V317       12
V318       12
V319       12
V320       12
V321       12
Length: 202, dtype: int64
0


In [57]:
# Separação de features e target
X = df.drop(['isFraud', 'TransactionID'], axis=1)
y = df['isFraud']

scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [58]:
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_scaled, y)

In [59]:
fig = px.bar(y_resampled.value_counts(), title='Distribuição das Classes - UnderSampler')
fig.show()

In [60]:
def create_time_windows_scaled(X_resampled, TransactionDT, time_steps=1):
    # Transforma TransactionDT de segundos para dias
    TransactionDT_days = TransactionDT / (60 * 60 * 24)
    TransactionDT_diff = np.diff(TransactionDT_days, prepend=TransactionDT_days[0])

    windows = []
    window_start = 0

    for i in range(1, len(X_resampled)):
        # Cria uma nova janela quando a diferença atinge o limite de time_steps (dias ou meses)
        if TransactionDT_diff[i] >= time_steps:
            windows.append(X_resampled[window_start:i])
            window_start = i

    if window_start < len(X_resampled):  # Adiciona a última janela
        windows.append(X_resampled[window_start:])

    return windows

windows = create_time_windows_scaled(X_resampled, df['TransactionDT'], time_steps=30)  # Para janelas mensais

In [61]:
def create_sequences_from_windows_scaled(windows, y, time_steps):
    X_seq, y_seq = [], []
    for window in windows:
        y_window = y[:len(window)]  # Assume que y tem o mesmo tamanho de X

        # Criar sequências dentro de cada janela
        for i in range(len(window) - time_steps):
            X_seq.append(window[i:i+time_steps])
            y_seq.append(y_window[i+time_steps])
    return np.array(X_seq), np.array(y_seq)

# Criação das sequências a partir dessas janelas
time_steps = 10  # Por exemplo, 10 transações consecutivas
X_seq, y_seq = create_sequences_from_windows_scaled(windows, y_resampled.values, time_steps)

In [62]:
print(X_seq.shape, y_seq.shape)

(41316, 10, 222) (41316,)


# Rede Neural LSTM

In [63]:
# Dividir os 20% de dados selecionados em treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

### Explicação dos Ajustes:
- As camadas `LSTM` intermediárias (100 e 50 unidades) têm `return_sequences=True`, o que significa que elas retornam a saída completa de cada célula ao longo do tempo. Isso garante que a próxima camada LSTM receba uma sequência tridimensional (3D) como entrada.
- A última camada `LSTM` (25 unidades) tem `return_sequences=False`, o que significa que apenas a última célula da sequência será passada para a camada densa (`Dense`), que é adequada para a previsão de uma única saída binária (fraude ou não fraude).
  
### Resumo:
- **Camadas intermediárias `LSTM`**: Devem ter `return_sequences=True` para garantir que passem sequências completas para a próxima camada LSTM.
- **Última camada `LSTM`**: Deve ter `return_sequences=False`, pois só precisamos da última saída para a camada densa final.

## Estrutura da arquitetura do modelo

In [64]:
  def create_model(X_train, optimizer=Adam, learn_rate=0.001):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))
    model.add(LSTM(100, return_sequences=True))  # Mantém todas as saídas
    model.add(LSTM(50, return_sequences=True))   # Mantém todas as saídas
    model.add(LSTM(25, return_sequences=False))  # Retorna apenas a última célula de saída
    model.add(Dense(1, activation='sigmoid'))    # Camada de saída para classificação binária

    model.compile(
        optimizer=optimizer(learning_rate=learn_rate),
        loss='binary_crossentropy',
        metrics=[Accuracy(), Precision(), Recall(), AUC()]
    )

    return model

In [65]:
model = create_model(X_train)
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                        │ (None, 10, 100)             │         129,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_9 (LSTM)                        │ (None, 10, 50)              │          30,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_10 (LSTM)                       │ (None, 25)                  │           7,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              26 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 167,026 (652.45 KB)

 Trainable params: 167,026 (652.45 KB)

 Non-trainable params: 0 (0.00 B)

## Treinamento do modelo

In [66]:
history = model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1, validation_data=(X_test, y_test))

Epoch 1/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.0000e+00 - auc_2: 0.9853 - loss: 0.1468 - precision_2: 0.9443 - recall_2: 0.9534 - val_accuracy: 0.0000e+00 - val_auc_2: 0.9986 - val_loss: 0.0338 - val_precision_2: 0.9859 - val_recall_2: 0.9900
Epoch 2/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0000e+00 - auc_2: 0.9995 - loss: 0.0240 - precision_2: 0.9932 - recall_2: 0.9908 - val_accuracy: 0.0000e+00 - val_auc_2: 0.9996 - val_loss: 0.0181 - val_precision_2: 0.9932 - val_recall_2: 0.9951
Epoch 3/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.0000e+00 - auc_2: 0.9998 - loss: 0.0127 - precision_2: 0.9956 - recall_2: 0.9956 - val_accuracy: 0.0000e+00 - val_auc_2: 0.9999 - val_loss: 0.0160 - val_precision_2: 0.9910 - val_recall_2: 0.9961
Epoch 4/10
517/517 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.0000e+00 - auc_2: 0.9999 - loss: 0.0096 - precision_2: 0.9963 - recall_2: 0.9965 - val_accuracy: 0.0000e+00 - val_auc_2: 0.9997 - val_loss: 0.0179 

In [80]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='loss'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='val_loss'))
fig.update_layout(title='Loss vs. Epochs', xaxis_title='Epochs', yaxis_title='Loss')
fig.show()

In [81]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['precision_2'], mode='lines', name='precision'))
fig.add_trace(go.Scatter(y=history.history['val_precision_2'], mode='lines', name='val_precision'))
fig.update_layout(title='Precision vs. Epochs', xaxis_title='Epochs', yaxis_title='Precision')
fig.show()

In [77]:
loss, accuracy, precision, recall, auc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, AUC: {roc_auc_score(y_test, model.predict(X_test))}')

259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0000e+00 - auc_2: 0.9999 - loss: 0.0055 - precision_2: 0.9972 - recall_2: 0.9998
259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Loss: 0.005972938612103462, Accuracy: 0.0, Precision: 0.9963601231575012, Recall: 0.9997565150260925, AUC: 0.9999870554324454


### Validação

In [78]:
y_pred = (model.predict(X_test) > 0.5).astype(int)

259/259 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [79]:
# Calculando métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)

# Imprimindo os resultados
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'AUC-ROC: {auc}')

Accuracy: 0.9980638915779284
Precision: 0.9963601067702014
Recall: 0.9997565132700268
AUC-ROC: 0.9980740709241642


# Análise dos resultados e comparações

Observando o gráfico de perda, percebi que a perda de treinamento diminui consistentemente a cada época, o que é esperado. No entanto, notei que a perda de validação começa a aumentar após algumas épocas, o que sugere overfitting. Isso indica que o modelo está se ajustando muito bem aos dados de treinamento, mas tem dificuldade em generalizar para dados novos.

Para lidar com esse overfitting, seria possível considerar algumas soluções:

- Adicionar regularização, como dropout ou L1/L2, nas camadas da rede.
- Aumentar o conjunto de dados de treinamento, se possível.
- Utilizar técnicas de aumento de dados (data augmentation) para gerar mais dados artificialmente.
- Reduzir a complexidade do modelo, diminuindo o número de camadas ou de neurônios.
- Implementar o early stopping, parando o treinamento quando a perda de validação começar a aumentar.

Além disso, seria benéfico otimizar os hiperparâmetros do modelo, como a taxa de aprendizado e o tamanho do lote, utilizando técnicas como GridSearchCV ou RandomizedSearchCV para buscar melhores resultados.

- Devido às colunas presentes no dataset, foi necessário realizar diversos tratamentos, como a remoção de valores NaN. Para os valores que permaneceram, apliquei um processo de imputação, substituindo os dados com base nos valores existentes, gerando dados artificiais.
- Isso acabou influenciando o treinamento do modelo e contribuiu para o overfitting.
- Portanto, seria preferível realizar uma curadoria mais rigorosa dos dados, para entender com precisão o que é realmente útil ou não.